In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [5]:
SIZE = 32
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(),
                transforms.Normalize((0.4918019, 0.48263696, 0.44733968), (0.24705184, 0.2433789, 0.26162848)),
                transforms.ToPILImage()
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE),
                transforms.ToTensor(),
                transforms.Normalize((0.4918019, 0.48263696, 0.44733968), (0.24705184, 0.2433789, 0.26162848)),
                transforms.ToPILImage()
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [6]:
# Training
batch_size = 128
train_imagefolder = LabImageFolder('../../datasets/cifar10/train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder('../../datasets/cifar10/val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [7]:
kernel_size=3
stride_en=2
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        
        self.convtrans1 = nn.ConvTranspose2d(32, 16, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(16, 8, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(8, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm8 = nn.BatchNorm2d(8)
        self.batchnorm16 = nn.BatchNorm2d(16)
        self.batchnorm32 = nn.BatchNorm2d(32)
        
        
    def forward(self, input):
        # encoder
        x = F.relu(self.batchnorm16(self.conv1(input)))
        x = F.relu(self.batchnorm32(self.conv2(x)))
        
        # decoder
        x = F.relu(self.batchnorm16(self.convtrans1(x)))
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.relu(self.batchnorm8(self.convtrans2(x)))
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [8]:
model = Autoencoder()

In [9]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [11]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [12]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             160
       BatchNorm2d-2           [-1, 16, 16, 16]              32
            Conv2d-3             [-1, 32, 8, 8]           4,640
       BatchNorm2d-4             [-1, 32, 8, 8]              64
   ConvTranspose2d-5             [-1, 16, 8, 8]           4,624
       BatchNorm2d-6             [-1, 16, 8, 8]              32
   ConvTranspose2d-7            [-1, 8, 16, 16]           1,160
       BatchNorm2d-8            [-1, 8, 16, 16]              16
   ConvTranspose2d-9            [-1, 2, 16, 16]             146
Total params: 10,874
Trainable params: 10,874
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.14
Params size (MB): 0.04
Estimated Total Size (MB): 0.19
---------------------------------------------

In [13]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [14]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'

In [15]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:        
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [17]:
# Make folders and set parameters
checkpoints = 'checkpoints'
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500

In [18]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.02241333 (0.03211824), PSNR 16.49493599 (15.66901112), SSIM 0.11696269 (0.09546477)
Finished training epoch 0
Validate: MSE 0.03066520 (0.03201322), PSNR 15.13354111 (14.95082111), SSIM 0.07589810 (0.09166242)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.02364009 (0.02330233), PSNR 16.26350975 (16.32729729), SSIM 0.15346698 (0.13573446)
Finished training epoch 1
Validate: MSE 0.03082146 (0.03223589), PSNR 15.11146736 (14.92153434), SSIM 0.07904083 (0.09533097)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.02220094 (0.02277298), PSNR 16.53628540 (16.42711611), SSIM 0.17445832 (0.16642855)
Finished training epoch 2
Validate: MSE 0.03347111 (0.03482657), PSNR 14.75329781 (14.58551211), SSIM 0.07771093 (0.09247864)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.02203953 (0.02250829), PSNR 16.56797600 (16.47761720), SSIM 0.19629548 (0.18189216)
Finished training epoch 3
Validate: MSE 0.03289222 (0.0

Epoch: 30, MSE 0.02102536 (0.02165194), PSNR 16.77256393 (16.64618596), SSIM 0.22327337 (0.21130271)
Finished training epoch 30
Validate: MSE 0.02269452 (0.02461862), PSNR 16.44078827 (16.09322092), SSIM 0.09215169 (0.11087213)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.02162061 (0.02162817), PSNR 16.65131950 (16.65091238), SSIM 0.20810251 (0.21165976)
Finished training epoch 31
Validate: MSE 0.02279462 (0.02459633), PSNR 16.42167473 (16.09748857), SSIM 0.08719370 (0.10747107)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.02169704 (0.02162512), PSNR 16.63599396 (16.65163607), SSIM 0.21003826 (0.21159536)
Finished training epoch 32
Validate: MSE 0.02271665 (0.02461019), PSNR 16.43655586 (16.09478684), SSIM 0.08726539 (0.10779579)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.02189905 (0.02161731), PSNR 16.59574699 (16.65329970), SSIM 0.21201062 (0.21185254)
Finished training epoch 33
Validate: MSE 0.02264981 (0.02449456), PSNR 

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return func(*args, **kwargs)


Validate: MSE 0.03088204 (0.03249368), PSNR 15.10293961 (14.88516004), SSIM 0.10318850 (0.11575704)
Finished validation.
Starting training epoch 42
Epoch: 42, MSE 0.02314260 (0.02155098), PSNR 16.35587692 (16.66659400), SSIM 0.21318543 (0.21336645)
Finished training epoch 42
Validate: MSE 0.03232783 (0.03414004), PSNR 14.90423393 (14.67049117), SSIM 0.10130484 (0.11408207)
Finished validation.
Starting training epoch 43
Epoch: 43, MSE 0.02171285 (0.02154377), PSNR 16.63282967 (16.66804971), SSIM 0.22092430 (0.21363133)
Finished training epoch 43
Validate: MSE 0.03345719 (0.03512554), PSNR 14.75510502 (14.54651050), SSIM 0.09724054 (0.10974282)
Finished validation.
Starting training epoch 44
Epoch: 44, MSE 0.02090146 (0.02155342), PSNR 16.79823303 (16.66608991), SSIM 0.22565815 (0.21349825)
Finished training epoch 44


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return func(*args, **kwargs)


...

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 9 pixels
  return func(*args, **kwargs)


Validate: MSE 0.04586123 (0.04747702), PSNR 13.38554287 (13.23680713), SSIM 0.08069552 (0.09381662)
Finished validation.
Starting training epoch 48
Epoch: 48, MSE 0.02141344 (0.02152569), PSNR 16.69313622 (16.67172822), SSIM 0.20532227 (0.21400772)
Finished training epoch 48
Validate: MSE 0.03755880 (0.03916655), PSNR 14.25288296 (14.07291153), SSIM 0.08513694 (0.09791299)
Finished validation.
Starting training epoch 49
Epoch: 49, MSE 0.02043425 (0.02151455), PSNR 16.89641190 (16.67386714), SSIM 0.21264446 (0.21424613)
Finished training epoch 49
Validate: MSE 0.03284813 (0.03493418), PSNR 14.83489323 (14.57012357), SSIM 0.08652408 (0.09821095)
Finished validation.
Starting training epoch 50
Epoch: 50, MSE 0.02002008 (0.02151657), PSNR 16.98534203 (16.67328286), SSIM 0.23235814 (0.21425676)
Finished training epoch 50
Validate: MSE 0.04427497 (0.04630731), PSNR 13.53841686 (13.34532221), SSIM 0.07588392 (0.08735249)
Finished validation.
Starting training epoch 51
Epoch: 51, MSE 0.0218816

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 11 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 7 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 14 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 6 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 12 pixels
  return func(*args, **kwargs)
/

Validate: MSE 0.06704713 (0.06929346), PSNR 11.73619747 (11.59459513), SSIM 0.06890370 (0.07782165)
Finished validation.
Starting training epoch 53
Epoch: 53, MSE 0.02217923 (0.02150086), PSNR 16.54053497 (16.67667597), SSIM 0.20036438 (0.21448729)
Finished training epoch 53
Validate: MSE 0.04870857 (0.05215629), PSNR 13.12394619 (12.82998097), SSIM 0.06944598 (0.07250472)
Finished validation.
Starting training epoch 54
Epoch: 54, MSE 0.02055471 (0.02149082), PSNR 16.87088585 (16.67867240), SSIM 0.21348815 (0.21463513)
Finished training epoch 54
Validate: MSE 0.05051012 (0.05417129), PSNR 12.96621513 (12.66619159), SSIM 0.06464631 (0.06871650)
Finished validation.
Starting training epoch 55
Epoch: 55, MSE 0.02107826 (0.02150731), PSNR 16.76165199 (16.67524210), SSIM 0.23417893 (0.21436564)
Finished training epoch 55
Validate: MSE 0.04675882 (0.04885203), PSNR 13.30136299 (13.11313262), SSIM 0.08123183 (0.09238637)
Finished validation.
Starting training epoch 56
Epoch: 56, MSE 0.0214134

<Figure size 432x288 with 0 Axes>

In [19]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [20]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.06157109 (0.06471485), PSNR 12.10623074 (11.89260266), SSIM 0.06743604 (0.06908165)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [21]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()